In [11]:
from supabase import create_client, Client
import os

supabase_url = os.getenv("SUPABASE_URL")
supabase_key = os.getenv("SUPABASE_KEY")

supabase: Client = create_client(supabase_url, supabase_key)

In [12]:
import pandas as pd

In [13]:
response = (
    supabase.table("Payments")
    .select("*")
    .csv()
    .execute()
)

In [14]:
response.data

'id,created_at,uuid,product_name,quantity,price,purchase_date,tags\n2,"2025-04-11 17:16:10.107133+00",47823327-2b0f-48c8-9513-614c3ab5d61a,test,10,100,2009-02-05,"{test,MegaTest}"\n3,"2025-04-11 17:17:29.596742+00",47823327-2b0f-48c8-9513-614c3ab5d61a,яблоко,10,30,2025-04-11,"{test,food}"\n5,"2025-04-11 17:18:17.36416+00",47823327-2b0f-48c8-9513-614c3ab5d61a,banana,20,200,2025-04-02,"{test,food}"\n6,"2025-04-11 17:18:33.797878+00",47823327-2b0f-48c8-9513-614c3ab5d61a,banana,15,1000,2025-04-11,"{food,test}"\n7,"2025-04-11 17:19:38.887289+00",47823327-2b0f-48c8-9513-614c3ab5d61a,banana,5,500,2025-04-12,"{test,food}"\n8,"2025-04-11 20:24:01.007393+00",47823327-2b0f-48c8-9513-614c3ab5d61a,Чипсы,10,150,2025-04-11,"{food,fastfood}"\n9,"2025-04-12 05:10:14.828025+00",47823327-2b0f-48c8-9513-614c3ab5d61a,Жвачка,2,50,2025-04-07,{}'

In [15]:
from io import StringIO

In [21]:
df = pd.read_csv(StringIO(response.data), index_col=0, parse_dates=['purchase_date'])
df.drop(columns=['created_at'], inplace=True)
df

,uuid,product_name,quantity,price,purchase_date,tags
id,,,,,,
2,47823327-2b0f-48c8-9513-614c3ab5d61a,test,10,100,2009-02-05,"{test,MegaTest}"
3,47823327-2b0f-48c8-9513-614c3ab5d61a,яблоко,10,30,2025-04-11,"{test,food}"
5,47823327-2b0f-48c8-9513-614c3ab5d61a,banana,20,200,2025-04-02,"{test,food}"
6,47823327-2b0f-48c8-9513-614c3ab5d61a,banana,15,1000,2025-04-11,"{food,test}"
7,47823327-2b0f-48c8-9513-614c3ab5d61a,banana,5,500,2025-04-12,"{test,food}"
8,47823327-2b0f-48c8-9513-614c3ab5d61a,Чипсы,10,150,2025-04-11,"{food,fastfood}"
9,47823327-2b0f-48c8-9513-614c3ab5d61a,Жвачка,2,50,2025-04-07,{}


In [22]:
df.dtypes

uuid                     object
product_name             object
quantity                  int64
price                     int64
purchase_date    datetime64[ns]
tags                     object
dtype: object

In [23]:
test_uuid = "47823327-2b0f-48c8-9513-614c3ab5d61a"

In [30]:
tmp = df[df['uuid'] == test_uuid][['product_name', 'quantity', 'price', 'purchase_date']]
tmp = tmp.groupby(['product_name', 'purchase_date']).sum().reset_index().sort_values(by='purchase_date')

tmp

,product_name,purchase_date,quantity,price
3,test,2009-02-05,10,100
0,banana,2025-04-02,20,200
4,Жвачка,2025-04-07,2,50
1,banana,2025-04-11,15,1000
5,Чипсы,2025-04-11,10,150
6,яблоко,2025-04-11,10,30
2,banana,2025-04-12,5,500


In [31]:
tmp['purchase_date'].unique().tolist()

[Timestamp('2009-02-05 00:00:00'),
 Timestamp('2025-04-02 00:00:00'),
 Timestamp('2025-04-07 00:00:00'),
 Timestamp('2025-04-11 00:00:00'),
 Timestamp('2025-04-12 00:00:00')]

In [35]:
import pandas as pd
import json

# Предположим, что tmp уже существует
# tmp = df[df['uuid'] == test_uuid][['product_name', 'quantity', 'price', 'purchase_date']]
# tmp = tmp.groupby(['product_name', 'purchase_date']).sum()

# Сброс индекса, чтобы сделать product_name и purchase_date обычными колонками
tmp_reset = tmp.reset_index()

# Создание списка уникальных дат
labels = tmp_reset['purchase_date'].astype(str).unique().tolist()

# Подготовка данных для Quantity
quantity_datasets = []
for product_name, group in tmp_reset.groupby('product_name'):
    quantity_datasets.append({
        "label": product_name,
        "data": group.set_index('purchase_date')['quantity'].reindex(labels, fill_value=0).tolist()
    })

# Подготовка данных для Price
price_datasets = []
for product_name, group in tmp_reset.groupby('product_name'):
    price_datasets.append({
        "label": product_name,
        "data": group.set_index('purchase_date')['price'].reindex(labels, fill_value=0).tolist()
    })

# Формирование финального JSON
chart_data = {
    "quantity": {
        "labels": labels,
        "datasets": quantity_datasets
    },
    "price": {
        "labels": labels,
        "datasets": price_datasets
    }
}

# Вывод JSON в читаемом формате
print(json.dumps(chart_data, indent=4))

{
    "quantity": {
        "labels": [
            "2009-02-05",
            "2025-04-02",
            "2025-04-07",
            "2025-04-11",
            "2025-04-12"
        ],
        "datasets": [
            {
                "label": "banana",
                "data": [
                    0,
                    20,
                    0,
                    15,
                    5
                ]
            },
            {
                "label": "test",
                "data": [
                    10,
                    0,
                    0,
                    0,
                    0
                ]
            },
            {
                "label": "\u0416\u0432\u0430\u0447\u043a\u0430",
                "data": [
                    0,
                    0,
                    2,
                    0,
                    0
                ]
            },
            {
                "label": "\u0427\u0438\u043f\u0441\u044b",
                "data": [
  